# Continuous Models in Fintech - Final Project

### Imports

In [86]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
from datetime import datetime

In [87]:
file_path = "data/final_project_data_04032024.csv"

In [88]:
bonds = pd.read_csv(file_path).drop(["call_date"],axis=1)

bonds['maturity_date'] = pd.to_datetime(bonds['maturity_date'], format='%m/%d/%Y')
today = datetime.today()
bonds['maturity'] = (bonds['maturity_date'] - today).dt.days / 365.25
bonds = bonds[bonds['maturity']>0]

In [90]:
threshold = 0.5

fixed_coupon_bonds = bonds[(bonds['security_type'].isin(['MARKET BASED NOTE', 'MARKET BASED BOND'])) & (bonds['rate'] > 0)]
fixed_face_value_bonds = bonds[~bonds['security_type'].isin(['TIPS'])]
close_to_par_bonds = bonds[abs(bonds['eod'] - 100) < threshold]

### Nelson Siegel

In [117]:
def nelson_siegel(params, maturities):
    beta0, beta1, beta2, tau = params
    return beta0 + beta1 * (1 - np.exp(-maturities / tau)) / (maturities / tau) + beta2 * ((1 - np.exp(-maturities / tau)) / (maturities / tau) - np.exp(-maturities / tau))


def loss_function_ns(params, maturities, yields):
    """
    MSE loss
    """
    model_yields = nelson_siegel(params, maturities)
    return np.sum((yields - model_yields) ** 2)


def calibrate_ns(bonds):
    maturities = bonds['maturity'].values
    yields = 100 / bonds['eod'].values - 1 

    initial_params = [0.03, -0.02, 0.02, 1.0]

    result = minimize(loss_function_ns, initial_params, args=(maturities, yields), method='BFGS')

    beta0, beta1, beta2, tau = result.x
    f = result.fun
    return beta0, beta1, beta2, tau,f

In [118]:
print("Calibrating Nelson-Siegel:\n")
beta0, beta1, beta2, tau,f = calibrate_ns(fixed_coupon_bonds)
beta0, beta1, beta2, tau = round(beta0,5),round(beta1,5),round(beta2,5),round(tau,5)
print(f'Calibrated parameters for fixed coupon bonds:\nbeta0 = {beta0}, beta1 = {beta1}, beta2 = {beta2}, tau = {tau}\nloss={f}\n')


beta0, beta1, beta2, tau,f = calibrate_ns(fixed_face_value_bonds)
beta0, beta1, beta2, tau = round(beta0,5),round(beta1,5),round(beta2,5),round(tau,5)
print(f'Calibrated parameters for fixed face value bonds:\nbeta0 = {beta0}, beta1 = {beta1}, beta2 = {beta2}, tau = {tau}\nloss={f}\n')

beta0, beta1, beta2, tau,f = calibrate_ns(close_to_par_bonds)
beta0, beta1, beta2, tau = round(beta0,5),round(beta1,5),round(beta2,5),round(tau,5)
print(f'Calibrated parameters for close to par bonds:\nbeta0 = {beta0}, beta1 = {beta1}, beta2 = {beta2}, tau = {tau}\nloss={f}\n')

Calibrating Nelson-Siegel:

Calibrated parameters for fixed coupon bonds:
beta0 = 1.9845, beta1 = -1.95198, beta2 = -1.92865, tau = 19.53341
loss=4.483597304830799

Calibrated parameters for fixed face value bonds:
beta0 = 2.15958, beta1 = -2.13158, beta2 = -2.04269, tau = 21.45655
loss=4.495163433302942

Calibrated parameters for close to par bonds:
beta0 = -0.0013, beta1 = 0.00236, beta2 = -0.0018, tau = 0.99994
loss=0.00016870487486863128


### Vasicek

r0 : volatility of the interest rate and in a way characterizes the amplitude of the instantaneous randomness inflow
beta : "long term mean level" All future trajectories of r will evolve around a mean level b in the long run
alpha : "speed of reversion" characterizes the velocity at which such trajectories will regroup around b in time
sigma : "instantaneous volatility", measures instant by instant the amplitude of randomness entering the system. higher values indicates more randomness

increasing sigma increases the amount of randomness entering the system, but at the same time increasing alpha amounts to increasing the speed at which the system will stabilize statistically around the long term mean beta with a corridor of variance determined also by alpha.

In [115]:
def vasicek(params, maturities):
    a, b, sigma, r0 = params
    return b + (r0 - b) * np.exp(-a * maturities) + sigma ** 2 / (2 * a) * (1 - np.exp(-2 * a * maturities))

def loss_function_vasicek(params, maturities, yields):
    """
    MSE loss
    """
    model_yields = vasicek(params, maturities)
    return np.sum((yields - model_yields) ** 2)

def calibrate_vasicek(bonds):
    maturities = bonds['maturity'].values
    yields = 100 / bonds['eod'].values - 1 

    initial_params = [1.0, 0.1, 0.2, 0.1]

    result = minimize(loss_function_vasicek, initial_params, args=(maturities, yields), method='BFGS')

    a, b, sigma, r0 = result.x
    f = result.fun
    return a, b, sigma, r0,f


In [116]:
print("Calibrating Vasicek:\n")
a, b, sigma, r0,f = calibrate_vasicek(fixed_coupon_bonds)
a,b,sigma,r0= round(a,5),round(b,5),round(sigma,5),round(r0,5)
print(f'Calibrated parameters for fixed coupon bonds (Vasicek):\na = {a}, b = {b}, sigma = {sigma}, r0 = {r0}\nloss={f}\n\n')

a, b, sigma, r0,f = calibrate_vasicek(fixed_face_value_bonds)
a,b,sigma,r0 = round(a,5),round(b,5),round(sigma,5),round(r0,5)
print(f'Calibrated parameters for fixed face value bonds (Vasicek):\na = {a}, b = {b}, sigma = {sigma}, r0 = {r0}\nloss={f}\n')

a, b, sigma, r0,f = calibrate_vasicek(close_to_par_bonds)
a,b,sigma,r0 = round(a,5),round(b,5),round(sigma,5),round(r0,5)
print(f'Calibrated parameters for close to par bonds (Vasicek):\na = {a}, b = {b}, sigma = {sigma}, r0 = {r0}\nloss={f}\n')

Calibrating Vasicek:

Calibrated parameters for fixed coupon bonds (Vasicek):
a = -0.00344, b = 33.4322, sigma = -0.34835, r0 = 0.02331
loss=4.503585280758019

Calibrated parameters for fixed face value bonds (Vasicek):
a = -0.00325, b = 36.44901, sigma = -0.3535, r0 = 0.02174
loss=4.511547970837192

Calibrated parameters for close to par bonds (Vasicek):
a = 1.01535, b = -0.00804, sigma = 0.11399, r0 = -0.00087
loss=0.00016398314475431967


### Hull-White

In [108]:
def hull_white(params, maturities):
    a, sigma = params
    return sigma * np.sqrt((1 - np.exp(-2 * a * maturities)) / (2 * a))

def loss_function_hw(params, maturities, yields):
    """
    MSE loss
    """
    model_yields = hull_white(params, maturities)
    return np.sum((yields - model_yields) ** 2)

def calibrate_hw(bonds):
    maturities = bonds['maturity'].values
    yields = 100 / bonds['eod'].values - 1 
    initial_params_hw = [0.1, 0.02]

    result_hw = minimize(loss_function_hw, initial_params_hw, args=(maturities, yields), method='BFGS')

    a, sigma = result_hw.x
    f = result_hw.fun
    return a, sigma,f

In [110]:
print("Calibrating Hull-White:\n")

a, sigma,f = calibrate_hw(fixed_coupon_bonds)
print(f'Calibrated parameters for fixed coupon bonds (Hull-White):\na={a}, sigma={sigma}\nloss={f}\n')

a, sigma,f = calibrate_hw(fixed_face_value_bonds)
print(f'Calibrated parameters for fixed face value bonds (Hull-White):\na={a}, sigma={sigma}\nloss={f}\n')

a, sigma,f = calibrate_hw(close_to_par_bonds)
print(f'Calibrated parameters for close to par bonds (Hull-White):\na={a}, sigma={sigma}\nloss={f}\n')

Calibrating Hull-White:

Calibrated parameters for fixed coupon bonds (Hull-White):
a=-0.05796157528184117, sigma=0.026546222111327032
loss=4.543005258545655

Calibrated parameters for fixed face value bonds (Hull-White):
a=-0.05800695307830862, sigma=0.02652441568934463
loss=4.552294074127071

Calibrated parameters for close to par bonds (Hull-White):
a=0.10253101840852562, sigma=-0.0005405023732157218
loss=0.00017254544990482043
